# Full RAG Pipeline Logger - Complete Flow Analysis

This notebook logs the complete RAG pipeline for both Format A and Format B:
- Vector database queries and responses
- RAG context building
- LLM prompts
- LLM responses
- Final answers

In [ ]:
import json
import logging
import sys
import os
from datetime import datetime
from typing import Dict, List, Any, Optional
import pandas as pd
from pinecone import Pinecone
import openai
import numpy as np

# Add project root to path
sys.path.append(os.path.dirname(os.path.abspath('')))

# Import utilities
from src.utils.embedding_client import create_embedding_client
from src.utils.token_manager import create_token_manager
from src.utils.binary_parser import parse_binary_response

print("Imports successful")

## Setup Logging Configuration

In [ ]:
# Create logs directory
log_dir = "rag_pipeline_logs"
os.makedirs(log_dir, exist_ok=True)

# Setup file loggers
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
log_filename = f"{log_dir}/full_pipeline_{timestamp}.log"
json_log_filename = f"{log_dir}/full_pipeline_{timestamp}.json"

# Configure logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_filename),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger('rag_pipeline_logger')
logger.info(f"Logging RAG pipeline to: {log_filename}")
logger.info(f"JSON logs will be saved to: {json_log_filename}")

## Enhanced RAG Classes with Full Pipeline Logging

In [ ]:
class FullPipelineLoggerFormatA:
    """Format A RAG with complete pipeline logging"""
    
    def __init__(self, config_path="experiments/config.json", model="openai"):
        with open(config_path, 'r') as f:
            self.config = json.load(f)
        
        # Initialize components
        self.pc = Pinecone(api_key=self.config['pinecone_api_key'])
        self.index = self.pc.Index(self.config['pinecone_index_name'])
        self.namespace = "drug-side-effects-formatA"
        
        # OpenAI for embeddings and LLM
        self.openai_client = openai.OpenAI(api_key=self.config['openai_api_key'])
        
        # Token manager
        self.token_manager = create_token_manager(model_type="openai")
        
        # Store all pipeline data
        self.pipeline_logs = []
        
        logger.info(f"✅ Format A Pipeline Logger initialized")
    
    def get_embedding(self, text: str) -> List[float]:
        """Generate embedding"""
        try:
            response = self.openai_client.embeddings.create(
                input=text,
                model="text-embedding-ada-002"
            )
            return response.data[0].embedding
        except Exception as e:
            logger.error(f"Embedding error: {e}")
            return None
    
    def query_with_full_logging(self, drug: str, side_effect: str) -> Dict[str, Any]:
        """Execute full RAG pipeline with detailed logging"""
        
        pipeline_data = {
            "format": "A",
            "query": {
                "drug": drug,
                "side_effect": side_effect,
                "timestamp": datetime.now().isoformat()
            }
        }
        
        logger.info("\n" + "="*80)
        logger.info(f"🔍 FORMAT A FULL PIPELINE: {drug} - {side_effect}")
        logger.info("="*80)
        
        # Step 1: Generate embedding
        query_text = f"{drug} {side_effect}"
        logger.info(f"\n📌 Step 1: Generating embedding for: '{query_text}'")
        query_embedding = self.get_embedding(query_text)
        
        if not query_embedding:
            logger.error("Failed to generate embedding")
            return None
        
        logger.info(f"✅ Embedding generated (dimension: {len(query_embedding)})")
        
        # Step 2: Query Pinecone
        logger.info(f"\n📌 Step 2: Querying Pinecone (namespace: {self.namespace})")
        
        results = self.index.query(
            vector=query_embedding,
            top_k=10,
            namespace=self.namespace,
            include_metadata=True
        )
        
        logger.info(f"✅ Retrieved {len(results.matches)} matches from Pinecone")
        
        # Log Pinecone matches
        pinecone_matches = []
        for i, match in enumerate(results.matches[:5], 1):  # Log top 5
            logger.info(f"   Match {i}: Score={match.score:.4f}, Drug={match.metadata.get('drug', 'N/A')}")
            pinecone_matches.append({
                "rank": i,
                "score": float(match.score),
                "drug": match.metadata.get('drug', ''),
                "text_preview": str(match.metadata.get('text', ''))[:200]
            })
        
        pipeline_data["pinecone_response"] = {
            "total_matches": len(results.matches),
            "top_matches": pinecone_matches
        }
        
        # Step 3: Build RAG context
        logger.info(f"\n📌 Step 3: Building RAG context from retrieved documents")
        
        context_documents = []
        for match in results.matches:
            if match.metadata and match.score > 0.5:
                drug_name = match.metadata.get('drug', '')
                drug_text = match.metadata.get('text', '')
                if drug_name and drug_text:
                    context_documents.append(f"Drug: {drug_name}\n{drug_text}")
        
        logger.info(f"✅ Built context from {len(context_documents)} relevant documents")
        
        # Token management and truncation
        base_prompt = f"""You are asked to answer the following question with a single word: YES or NO. Base your answer strictly on the RAG Results provided below. After your YES or NO answer, briefly explain your reasoning using the information from the RAG Results. Do not infer or speculate beyond the provided information.

### Question:

Is {side_effect} an adverse effect of {drug}?

### RAG Results:

{{context}}"""
        
        if context_documents:
            context, docs_included = self.token_manager.truncate_context_documents(context_documents, base_prompt)
            logger.info(f"📊 Context truncation: {docs_included}/{len(context_documents)} documents included")
        else:
            context = f"No data found for {drug}"
            docs_included = 0
        
        # Save RAG context
        pipeline_data["rag_context"] = {
            "total_documents": len(context_documents),
            "documents_included": docs_included,
            "context_preview": context[:500] + "..." if len(context) > 500 else context
        }
        
        # Step 4: Build final prompt
        prompt = base_prompt.format(context=context)
        
        logger.info(f"\n📌 Step 4: Sending prompt to LLM")
        logger.info(f"Prompt length: {len(prompt)} characters")
        logger.info(f"\n--- PROMPT SENT TO LLM ---\n{prompt[:500]}...\n--- END PROMPT PREVIEW ---")
        
        pipeline_data["llm_prompt"] = {
            "full_prompt_length": len(prompt),
            "prompt_preview": prompt[:1000]
        }
        
        # Step 5: Get LLM response
        try:
            response = self.openai_client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=100,
                temperature=0.1
            )
            
            llm_response = response.choices[0].message.content
            
            logger.info(f"\n📌 Step 5: Received LLM response")
            logger.info(f"\n--- LLM RESPONSE ---\n{llm_response}\n--- END LLM RESPONSE ---")
            
            # Step 6: Parse answer
            answer = parse_binary_response(llm_response)
            
            logger.info(f"\n📌 Step 6: Parsed final answer: {answer}")
            
            pipeline_data["llm_response"] = {
                "raw_response": llm_response,
                "parsed_answer": answer,
                "confidence": 0.9 if answer != 'UNKNOWN' else 0.3
            }
            
            # Store complete pipeline log
            self.pipeline_logs.append(pipeline_data)
            
            logger.info(f"\n✅ FORMAT A PIPELINE COMPLETE: {drug} + {side_effect} = {answer}")
            logger.info("="*80)
            
            return pipeline_data
            
        except Exception as e:
            logger.error(f"LLM error: {e}")
            pipeline_data["error"] = str(e)
            self.pipeline_logs.append(pipeline_data)
            return pipeline_data
    
    def save_logs(self):
        """Save all pipeline logs to JSON"""
        with open(json_log_filename, 'w') as f:
            json.dump(self.pipeline_logs, f, indent=2)
        logger.info(f"\n💾 Saved {len(self.pipeline_logs)} pipeline logs to {json_log_filename}")
        return json_log_filename

In [ ]:
class FullPipelineLoggerFormatB:
    """Format B RAG with complete pipeline logging"""
    
    def __init__(self, config_path="experiments/config.json", model="openai"):
        with open(config_path, 'r') as f:
            self.config = json.load(f)
        
        # Initialize components
        self.pc = Pinecone(api_key=self.config['pinecone_api_key'])
        self.index = self.pc.Index(self.config['pinecone_index_name'])
        self.namespace = "drug-side-effects-formatB"
        
        # OpenAI for embeddings and LLM
        self.openai_client = openai.OpenAI(api_key=self.config['openai_api_key'])
        
        # Token manager
        self.token_manager = create_token_manager(model_type="openai")
        
        # Store all pipeline data
        self.pipeline_logs = []
        
        logger.info(f"✅ Format B Pipeline Logger initialized")
    
    def get_embedding(self, text: str) -> List[float]:
        """Generate embedding"""
        try:
            response = self.openai_client.embeddings.create(
                input=text,
                model="text-embedding-ada-002"
            )
            return response.data[0].embedding
        except Exception as e:
            logger.error(f"Embedding error: {e}")
            return None
    
    def query_with_full_logging(self, drug: str, side_effect: str) -> Dict[str, Any]:
        """Execute full RAG pipeline with detailed logging"""
        
        pipeline_data = {
            "format": "B",
            "query": {
                "drug": drug,
                "side_effect": side_effect,
                "timestamp": datetime.now().isoformat()
            }
        }
        
        logger.info("\n" + "="*80)
        logger.info(f"🔍 FORMAT B FULL PIPELINE: {drug} - {side_effect}")
        logger.info("="*80)
        
        # Step 1: Generate embedding
        query_text = f"{drug} {side_effect}"
        logger.info(f"\n📌 Step 1: Generating embedding for: '{query_text}'")
        query_embedding = self.get_embedding(query_text)
        
        if not query_embedding:
            logger.error("Failed to generate embedding")
            return None
        
        logger.info(f"✅ Embedding generated (dimension: {len(query_embedding)})")
        
        # Step 2: Query Pinecone
        logger.info(f"\n📌 Step 2: Querying Pinecone (namespace: {self.namespace})")
        
        results = self.index.query(
            vector=query_embedding,
            top_k=10,
            namespace=self.namespace,
            include_metadata=True
        )
        
        logger.info(f"✅ Retrieved {len(results.matches)} matches from Pinecone")
        
        # Log Pinecone matches
        pinecone_matches = []
        drug_relevant_count = 0
        
        for i, match in enumerate(results.matches[:5], 1):  # Log top 5
            pair_drug = match.metadata.get('drug', '')
            pair_effect = match.metadata.get('side_effect', '')
            is_relevant = drug.lower() in pair_drug.lower()
            
            if is_relevant:
                drug_relevant_count += 1
                
            logger.info(f"   Match {i}: Score={match.score:.4f}, {pair_drug} → {pair_effect} {'✓' if is_relevant else '✗'}")
            
            pinecone_matches.append({
                "rank": i,
                "score": float(match.score),
                "drug": pair_drug,
                "side_effect": pair_effect,
                "is_drug_relevant": is_relevant
            })
        
        pipeline_data["pinecone_response"] = {
            "total_matches": len(results.matches),
            "drug_relevant_matches": drug_relevant_count,
            "top_matches": pinecone_matches
        }
        
        # Step 3: Build RAG context (drug-effect pairs)
        logger.info(f"\n📌 Step 3: Building RAG context from drug-effect pairs")
        
        context_pairs = []
        for match in results.matches:
            if match.metadata and match.score > 0.5:
                pair_drug = match.metadata.get('drug', '')
                pair_effect = match.metadata.get('side_effect', '')
                # Filter for relevant drug
                if pair_drug and pair_effect and drug.lower() in pair_drug.lower():
                    context_pairs.append(f"• {pair_drug} → {pair_effect}")
        
        logger.info(f"✅ Built context from {len(context_pairs)} drug-relevant pairs")
        
        # Build prompt
        base_prompt = f"""You are asked to answer the following question with a single word: YES or NO.

The RAG Results below show drug-side effect relationships where "Drug → Side Effect" means the drug causes that side effect as an adverse reaction.

Instructions:
- Answer YES if the RAG Results show that {drug} causes {side_effect} as an adverse reaction
- Answer NO if the RAG Results do not show this relationship or show no relevant information
- You must start your response with either YES or NO

### Question:

Is {side_effect} an adverse effect of {drug}?

### RAG Results:

{{context}}"""
        
        if context_pairs:
            context, pairs_included = self.token_manager.truncate_context_pairs(context_pairs, base_prompt)
            logger.info(f"📊 Context truncation: {pairs_included}/{len(context_pairs)} pairs included")
        else:
            context = f"No specific pairs found for {drug} and {side_effect}"
            pairs_included = 0
        
        # Save RAG context
        pipeline_data["rag_context"] = {
            "total_pairs": len(context_pairs),
            "pairs_included": pairs_included,
            "context_preview": context[:500] + "..." if len(context) > 500 else context
        }
        
        # Step 4: Build final prompt
        prompt = base_prompt.format(context=context)
        
        logger.info(f"\n📌 Step 4: Sending prompt to LLM")
        logger.info(f"Prompt length: {len(prompt)} characters")
        logger.info(f"\n--- PROMPT SENT TO LLM ---\n{prompt[:500]}...\n--- END PROMPT PREVIEW ---")
        
        pipeline_data["llm_prompt"] = {
            "full_prompt_length": len(prompt),
            "prompt_preview": prompt[:1000]
        }
        
        # Step 5: Get LLM response
        try:
            response = self.openai_client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=100,
                temperature=0.1
            )
            
            llm_response = response.choices[0].message.content
            
            logger.info(f"\n📌 Step 5: Received LLM response")
            logger.info(f"\n--- LLM RESPONSE ---\n{llm_response}\n--- END LLM RESPONSE ---")
            
            # Step 6: Parse answer
            answer = parse_binary_response(llm_response)
            
            logger.info(f"\n📌 Step 6: Parsed final answer: {answer}")
            
            pipeline_data["llm_response"] = {
                "raw_response": llm_response,
                "parsed_answer": answer,
                "confidence": 0.9 if answer != 'UNKNOWN' else 0.3
            }
            
            # Store complete pipeline log
            self.pipeline_logs.append(pipeline_data)
            
            logger.info(f"\n✅ FORMAT B PIPELINE COMPLETE: {drug} + {side_effect} = {answer}")
            logger.info("="*80)
            
            return pipeline_data
            
        except Exception as e:
            logger.error(f"LLM error: {e}")
            pipeline_data["error"] = str(e)
            self.pipeline_logs.append(pipeline_data)
            return pipeline_data
    
    def save_logs(self):
        """Save all pipeline logs to JSON"""
        with open(json_log_filename, 'w') as f:
            json.dump(self.pipeline_logs, f, indent=2)
        logger.info(f"\n💾 Saved {len(self.pipeline_logs)} pipeline logs to {json_log_filename}")
        return json_log_filename

## Initialize Pipeline Loggers

In [ ]:
# Initialize both format loggers
format_a_logger = FullPipelineLoggerFormatA(config_path="experiments/config.json")
format_b_logger = FullPipelineLoggerFormatB(config_path="experiments/config.json")

print("✅ Pipeline loggers initialized")
print(f"\nLog files:")
print(f"  - Text log: {log_filename}")
print(f"  - JSON log: {json_log_filename}")

## Test Queries - Format A Full Pipeline

In [ ]:
# Define test queries
test_queries = [
    {"drug": "aspirin", "side_effect": "headache"},
    {"drug": "ibuprofen", "side_effect": "nausea"},
    {"drug": "metformin", "side_effect": "dizziness"},
]

print("\n" + "="*80)
print("🚀 RUNNING FORMAT A FULL PIPELINE TESTS")
print("="*80)

format_a_results = []
for query in test_queries:
    result = format_a_logger.query_with_full_logging(
        drug=query["drug"],
        side_effect=query["side_effect"]
    )
    if result:
        format_a_results.append(result)
        answer = result.get('llm_response', {}).get('parsed_answer', 'ERROR')
        print(f"\n✅ Format A: {query['drug']} + {query['side_effect']} = {answer}")

## Test Queries - Format B Full Pipeline

In [ ]:
print("\n" + "="*80)
print("🚀 RUNNING FORMAT B FULL PIPELINE TESTS")
print("="*80)

format_b_results = []
for query in test_queries:
    result = format_b_logger.query_with_full_logging(
        drug=query["drug"],
        side_effect=query["side_effect"]
    )
    if result:
        format_b_results.append(result)
        answer = result.get('llm_response', {}).get('parsed_answer', 'ERROR')
        print(f"\n✅ Format B: {query['drug']} + {query['side_effect']} = {answer}")

## Analyze Pipeline Components

In [ ]:
def analyze_pipeline_results(results, format_name):
    """Analyze the complete pipeline flow"""
    print(f"\n{'='*60}")
    print(f"📊 PIPELINE ANALYSIS: {format_name}")
    print(f"{'='*60}")
    
    for result in results:
        query = result['query']
        print(f"\n🔍 Query: {query['drug']} - {query['side_effect']}")
        
        # Pinecone stats
        if 'pinecone_response' in result:
            pr = result['pinecone_response']
            print(f"\n  📌 Vector Search:")
            print(f"     Total matches: {pr['total_matches']}")
            if 'drug_relevant_matches' in pr:
                print(f"     Drug-relevant: {pr['drug_relevant_matches']}")
            if pr['top_matches']:
                print(f"     Top score: {pr['top_matches'][0]['score']:.4f}")
        
        # RAG context stats
        if 'rag_context' in result:
            rc = result['rag_context']
            print(f"\n  📚 RAG Context:")
            if 'total_documents' in rc:
                print(f"     Documents: {rc['documents_included']}/{rc['total_documents']}")
            elif 'total_pairs' in rc:
                print(f"     Pairs: {rc['pairs_included']}/{rc['total_pairs']}")
            print(f"     Context preview: {rc['context_preview'][:100]}...")
        
        # LLM prompt stats
        if 'llm_prompt' in result:
            lp = result['llm_prompt']
            print(f"\n  📝 LLM Prompt:")
            print(f"     Length: {lp['full_prompt_length']} chars")
        
        # Final answer
        if 'llm_response' in result:
            lr = result['llm_response']
            print(f"\n  ✅ Final Answer: {lr['parsed_answer']}")
            print(f"     Confidence: {lr['confidence']}")
            print(f"     Response preview: {lr['raw_response'][:100]}...")
        
        print(f"\n  " + "-"*56)

# Analyze both formats
analyze_pipeline_results(format_a_results, "Format A")
analyze_pipeline_results(format_b_results, "Format B")

## Save All Pipeline Logs

In [ ]:
# Save all logs to JSON files
all_logs = {
    "format_a": format_a_logger.pipeline_logs,
    "format_b": format_b_logger.pipeline_logs,
    "metadata": {
        "timestamp": datetime.now().isoformat(),
        "config_file": "experiments/config.json",
        "total_queries": len(format_a_logger.pipeline_logs) + len(format_b_logger.pipeline_logs)
    }
}

with open(json_log_filename, 'w') as f:
    json.dump(all_logs, f, indent=2)

print(f"\n💾 Saved complete pipeline logs to: {json_log_filename}")
print(f"\nLog files contain:")
print(f"  - Vector DB queries and responses")
print(f"  - RAG context building")
print(f"  - Full prompts sent to LLM")
print(f"  - LLM responses")
print(f"  - Final parsed answers")

## Custom Query Testing with Full Pipeline Logging

In [ ]:
# Test your own custom queries
custom_drug = "aspirin"  # Change this
custom_side_effect = "bleeding"  # Change this

print(f"\n🔍 Testing custom query: {custom_drug} - {custom_side_effect}")
print("="*60)

# Test Format A
print("\n📌 Format A Full Pipeline:")
format_a_custom = format_a_logger.query_with_full_logging(custom_drug, custom_side_effect)

# Test Format B
print("\n📌 Format B Full Pipeline:")
format_b_custom = format_b_logger.query_with_full_logging(custom_drug, custom_side_effect)

# Display results summary
if format_a_custom and format_b_custom:
    print("\n" + "="*60)
    print("📊 CUSTOM QUERY RESULTS SUMMARY")
    print("="*60)
    
    a_answer = format_a_custom.get('llm_response', {}).get('parsed_answer', 'ERROR')
    b_answer = format_b_custom.get('llm_response', {}).get('parsed_answer', 'ERROR')
    
    print(f"\nQuery: Is {custom_side_effect} an adverse effect of {custom_drug}?")
    print(f"\nFormat A Answer: {a_answer}")
    print(f"Format B Answer: {b_answer}")
    
    if a_answer == b_answer:
        print(f"\n✅ Both formats agree: {a_answer}")
    else:
        print(f"\n⚠️ Formats disagree: A={a_answer}, B={b_answer}")

## Display Sample Pipeline Flow

In [ ]:
# Display a complete pipeline flow for examination
if format_a_logger.pipeline_logs:
    print("\n" + "="*60)
    print("📋 SAMPLE COMPLETE PIPELINE FLOW (Format A, First Query)")
    print("="*60)
    
    sample = format_a_logger.pipeline_logs[0]
    
    print(f"\n1️⃣ QUERY: {sample['query']['drug']} - {sample['query']['side_effect']}")
    
    print(f"\n2️⃣ VECTOR SEARCH RESULTS:")
    print(json.dumps(sample['pinecone_response'], indent=2)[:500])
    
    print(f"\n3️⃣ RAG CONTEXT:")
    print(json.dumps(sample['rag_context'], indent=2))
    
    print(f"\n4️⃣ LLM PROMPT (first 500 chars):")
    print(sample['llm_prompt']['prompt_preview'][:500])
    
    print(f"\n5️⃣ LLM RESPONSE:")
    print(sample['llm_response']['raw_response'])
    
    print(f"\n6️⃣ FINAL ANSWER: {sample['llm_response']['parsed_answer']}")